# 1. Project Goal/Motivation

## Goal:
Develop an image classification model that can accurately identify different species of bears from images.

## Motivation:
- **Problem**: Different bear species exhibit different behaviors, and appropriate human responses to encounters with them vary. Accurate identification can help in promoting safety and effective wildlife management.
- **Relevance**: This project can aid hikers, wildlife enthusiasts, and conservationists in identifying bear species quickly and accurately, contributing to safer interactions and better conservation strategies.


# 2. Data

## Data Collection:
- The dataset for this project consists of images of various bear species sourced from a publicly available dataset.

## Categories:
- The dataset includes images of common bear species such as:
  1. Polar bears
  2. Grizzly bears
  3. Black bears
  4. Panda bears
  5. Teddy bears

## Data Preparation:
- Image preprocessing steps were applied to ensure consistency and quality:
  - Image resizing: Images were resized to a standardized dimension (e.g., 224x224 pixels).
  - Data augmentation: Techniques like rotation, zoom, and flipping were applied to increase dataset variability and improve model robustness.



In [2]:
# Importing necessary libraries for model

# Building deep learning models
import tensorflow as tf
from tensorflow import keras
# For accessing pre-trained models
import tensorflow_hub as hub
# For separating train and test sets
from sklearn.model_selection import train_test_split

# For visualizations
import matplotlib.pyplot as plt
import matplotlib.image as img
import PIL.Image as Image
import cv2

import os
import numpy as np
import pandas as pd
import pathlib

In [3]:
# Accessing the images link
data_dir = "./input/bear-dataset/data" # Datasets path
data_dir = pathlib.Path(data_dir)
data_dir

# Opening each folder in a variable
black = list(data_dir.glob('black/*'))
grizzly = list(data_dir.glob('grizzly/*'))
panda = list(data_dir.glob('panda/*'))
polar = list(data_dir.glob('polar/*'))
teddy = list(data_dir.glob('teddy/*'))

# Assigning dirs for images and their labels
# Contains the images path
df_images = {
    'black' : black,
    'grizzly' : grizzly,
    'panda' : panda,
    'polar' : polar,
    'teddy': teddy
}

# Contains numerical labels for the categories
df_labels = {
    'black' : 0,
    'grizzly' : 1,
    'panda' : 2,
    'polar' : 3,
    'teddy': 4
}

# Reshape dimensions 224x224
X, y = [], [] # X = images, y = labels
for label, images in df_images.items():
    for image in images:
        img = cv2.imread(str(image))
        resized_img = cv2.resize(img, (224, 224)) # Resizing the images to be able to pass on MobileNetv2 model
        X.append(resized_img)
        y.append(df_labels[label])

# Standarizing
X = np.array(X)
X = X/255
y = np.array(y)

# Training and Test Dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# 3. Model

### TensorFlow Hub - MobileNetV2
In this section of the notebook, we are utilizing a pre-trained MobileNetV2 model from TensorFlow Hub as the base for our neural network. MobileNetV2 is a lightweight, efficient convolutional neural network commonly used for feature extraction in image classification tasks. By importing the model without its final classification layer, we can leverage its pre-trained weights to extract high-level features from our input images.

We initialize the MobileNetV2 model as a non-trainable Keras layer to retain its pre-trained weights during our training process. This approach, known as transfer learning, allows us to build a more effective model with reduced computational cost and training time, especially beneficial when working with smaller datasets.

We then construct a new sequential model by adding a dense layer on top of the MobileNetV2 base. This dense layer, configured to match the number of target classes in our specific task, will be trained to perform the final classification.

By compiling and summarizing this model, we prepare it for subsequent training and evaluation, effectively customizing a state-of-the-art image classification model for our unique dataset.

We compile the model using the Adam optimizer and `SparseCategoricalCrossentropy` loss function, with accuracy as the evaluation metric. The model is trained on the training dataset for 10 epochs to adjust its weights and minimize the loss. This process generates a history object that tracks the model's performance and learning progression over the epochs.

In [4]:
mobile_net = 'https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4' # MobileNetv4 link
mobile_net = hub.KerasLayer(
        mobile_net, input_shape=(224,224, 3), trainable=False) # Removing the last layer

In [5]:
num_label = 5 # number of labels

# Wrap the mobile_net layer in a Lambda layer to make it compatible with Sequential
model = keras.Sequential([
    keras.layers.Lambda(lambda x: mobile_net(x)),
    keras.layers.Dense(num_label)
])

model.summary()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lambda (Lambda)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [6]:
# Training the model
model.compile(
  optimizer="adam",
  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['acc'])


history = model.fit(X_train, y_train, epochs=10)

Epoch 1/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 3s 159ms/step - acc: 0.3511 - loss: 1.5380
Epoch 2/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 154ms/step - acc: 0.8055 - loss: 0.6438
Epoch 3/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 156ms/step - acc: 0.9276 - loss: 0.3276
Epoch 4/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 180ms/step - acc: 0.9274 - loss: 0.2217
Epoch 5/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 2s 212ms/step - acc: 0.9837 - loss: 0.1326
Epoch 6/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 2s 209ms/step - acc: 0.9986 - loss: 0.1000
Epoch 7/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 167ms/step - acc: 1.0000 - loss: 0.0900
Epoch 8/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 156ms/step - acc: 1.0000 - loss: 0.0669
Epoch 9/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 157ms/step - acc: 1.0000 - loss: 0.0524
Epoch 10/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 157ms/step - acc: 1.0000 - loss: 0.0562


# 4. Interpretation and Validation

We evaluate the model's performance on the test dataset to obtain its accuracy and other metrics. Subsequently, we generate predictions for the test data and convert these predictions to class labels. Finally, we print a detailed classification report, including precision, recall, and F1-score for each class, using the true labels and predicted labels.

These results indicate that the model achieved a relatively low loss of 0.1740 and a high accuracy of 91.94% during training. The evaluation on the test dataset confirms the model's performance, with a similar loss of 0.17397 and accuracy of 91.93%.

The classification report provides more detailed insights into the model's performance across different classes. Notably, class 0 has perfect precision but lower recall, suggesting that the model correctly identifies instances of this class but may miss some. Class 1 also has high precision and recall, indicating good performance. However, class 2 shows lower precision and recall, suggesting potential challenges in correctly classifying instances of this class. Classes 3 and 4 have high precision and recall, indicating that the model performs well on these classes.

Overall, the model achieves an accuracy of 92% on the test dataset, with a balanced performance across most classes. However, class 2 appears to be more challenging for the model, possibly due to imbalanced data or inherent difficulties in distinguishing instances of this class. Further analysis, such as investigating misclassified instances or exploring different model architectures, may help improve the model's performance, particularly for class 2.

In [7]:
model.evaluate(X_test,y_test)

2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 160ms/step - acc: 0.9681 - loss: 0.1049


[0.106674924492836, 0.9677419066429138]

In [8]:
from sklearn.metrics import classification_report

y_pred = model.predict(X_test, batch_size=64, verbose=1)
y_pred_bool = np.argmax(y_pred, axis=1)

print(classification_report(y_test, y_pred_bool))

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 558ms/step
              precision    recall  f1-score   support

           0       1.00      0.93      0.96        14
           1       1.00      1.00      1.00         8
           2       0.90      1.00      0.95         9
           3       0.95      1.00      0.97        19
           4       1.00      0.92      0.96        12

    accuracy                           0.97        62
   macro avg       0.97      0.97      0.97        62
weighted avg       0.97      0.97      0.97        62

